# Определение тональности отзывов на банки с помощью классических алгоритмов машинного обучения

Используем логистическую регрессию и мешок слов.



In [ ]:
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 41.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Константы

In [ ]:
max_words = 10000
random_state = 42

## Загружаем и готовим набор данных

In [ ]:
!wget https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1 -O banks.csv

--2023-10-21 06:56:09--  https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/a9r0b2yj3vqvi13/banks.csv [following]
--2023-10-21 06:56:10--  https://www.dropbox.com/s/dl/a9r0b2yj3vqvi13/banks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc736b0b56a94342d5e2ddb75767.dl.dropboxusercontent.com/cd/0/get/CGAoSbSyZBow9JEPmKWmgMZ1YnQgpyvSpB2KMTksjpQA9ISv4W0Floehk96YXbXzW3GTonNq1yGyq-j0RMML6c1C_Smmd2VwXPy1_ZkR_sFwU74PcI0HysLV8Sn05N99EaJ4hm0F712nhM1lmgCAhwtp/file?dl=1# [following]
--2023-10-21 06:56:10--  https://uc736b0b56a94342d5e2ddb75767.dl.dropboxusercontent.com/cd/0/get/CGAoSbSyZBow9JEPmKWmgMZ1YnQgpyvSpB2KMTksjpQA9ISv4W0Floehk96YXbXzW3GTonNq1yGyq-j0RMML6c1C_Smmd2VwXPy1_ZkR_sFwU

In [ ]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx');

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()

In [ ]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[62, 43, 916, 78, 204, 1867, 357, 2710, 409, 1..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[267, 855, 1178, 3214, 3891, 2981, 7345, 171, ..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 475, 183, 3, 628, 1768,..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4817, 72, 2368, 9, 62, 18, 2664, 1287, 3..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[418, 3850, 275, 87, 2, 648, 3, 4893, 2, 296, ..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[401, 2321, 809, 5027, 8157, 7, 3098, 126, 441..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[362, 24, 9494, 2151, 201, 14, 24, 95, 141, 66..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 109, 18, 992, 53, 38, 224, 233, 3, 46, 3..."


Считаем частоту слов во всех отзывах

In [ ]:
words = Counter()

In [ ]:
for txt in banks['Preprocessed_texts']:
    words.update(txt)

Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:
- Код заполнитель: 0
- Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [ ]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

Создаем словари

In [ ]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

Функция для преобразования списка слов в список кодов

In [ ]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

Преобразуем все тексты в последовательность кодов слов

In [ ]:
banks['Sequences'] = banks.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[62, 43, 916, 78, 204, 1867, 357, 2710, 409, 1..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[267, 855, 1178, 3214, 3891, 2981, 7345, 171, ..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 475, 183, 3, 628, 1768,..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4817, 72, 2368, 9, 62, 18, 2664, 1287, 3..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[418, 3850, 275, 87, 2, 648, 3, 4893, 2, 296, ..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[401, 2321, 809, 5027, 8157, 7, 3098, 126, 441..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[362, 24, 9494, 2151, 201, 14, 24, 95, 141, 66..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 109, 18, 992, 53, 38, 224, 233, 3, 46, 3..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [ ]:
mapping = {'Negative': 0, 'Positive': 1}

In [ ]:
banks.replace({'Score': mapping}, inplace=True)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[62, 43, 916, 78, 204, 1867, 357, 2710, 409, 1..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[267, 855, 1178, 3214, 3891, 2981, 7345, 171, ..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 475, 183, 3, 628, 1768,..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4817, 72, 2368, 9, 62, 18, 2664, 1287, 3..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[418, 3850, 275, 87, 2, 648, 3, 4893, 2, 296, ..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[401, 2321, 809, 5027, 8157, 7, 3098, 126, 441..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[362, 24, 9494, 2151, 201, 14, 24, 95, 141, 66..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 109, 18, 992, 53, 38, 224, 233, 3, 46, 3..."


### Выделяем данные для обучения и тестирования

In [ ]:
train, test = train_test_split(banks, test_size=0.2)

In [ ]:
train

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
13024,0,Вообще банк этот очень клиентоорентирован. Ког...,"[вообще, банк, очень, клиентоорентировать, я, ...","[138, 2, 18, 90, 19, 699, 9, 1520, 641, 353, 3..."
6213,1,...пока не удавалось.А про БИН-банк (дополните...,"[пока, удавалось., бин-банк, дополнительный, о...","[153, 2408, 273, 24, 4711, 1972, 39, 15, 115, ..."
4859,1,Я клиент банка в течение 3-х лет. Началось все...,"[клиент, банк, течение, 3-й, год, начаться, ав...","[13, 2, 107, 920, 14, 414, 729, 91, 52, 137, 7..."
10463,1,Добрый день! С тех пор как в нашем городе появ...,"[добрый, день, тот, пора, наш, город, появитьс...","[181, 5, 106, 303, 133, 291, 497, 344, 1076, 6..."
1006,1,Хочу поделиться своим положительным опытом раб...,"[хотеть, поделиться, свой, положительный, опыт...","[39, 999, 17, 203, 449, 29, 2472, 2, 13, 2, 11..."
...,...,...,...,...
4070,1,Пользуюсь услугами банка Русский Стандарт уже ...,"[пользоваться, услуга, банк, русский, стандарт...","[77, 44, 2, 674, 691, 397, 438, 4, 446, 9, 8, ..."
881,0,"Сегодня, посмотрев выписку в ИБ обнаружил таку...","[сегодня, посмотреть, выписка, иб, обнаружить,...","[109, 436, 255, 847, 613, 25, 3944, 5877, 1163..."
12884,1,"Ребята, у вас отличный банк! Это проявилось им...","[ребята, отличный, банк, это, проявиться, имен...","[1328, 604, 2, 4, 8411, 172, 113, 133, 5874, 1..."


In [ ]:
test

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
10241,1,У моей конторы (моей в смысле я хозяин и ген.д...,"[мой, контора, мой, смысл, хозяин, ген.дир, сч...","[6, 2007, 6, 996, 5584, 11, 4532, 344, 18, 128..."
9612,0,В ЛК интернет-банка «Альфа-Клик» воспользовалс...,"[лк, интернет-банка, альфа-клик, воспользовать...","[5154, 552, 894, 437, 150, 30, 1063, 341, 115,..."
2998,0,"27 марта 2014 г. было оставлено заявление, осп...","[27, март, 2014, г., оставить, заявление, оспа...","[1381, 321, 635, 126, 207, 27, 3793, 80, 3, 17..."
11176,1,Пришел сегодня в допофис №6901/01668 Сбербанка...,"[прийти, сегодня, допофис, №6901/01668, сберба...","[46, 109, 2612, 143, 126, 197, 1315, 43, 1819,..."
10527,1,В феврале прошлого года взяла кредит в Русфина...,"[февраль, прошлое, год, взять, кредит, русфина...","[468, 1587, 14, 127, 9, 3102, 267, 32, 100, 36..."
...,...,...,...,...
6135,1,С Хоум Кредитом работаю только полгода. Впервы...,"[хоум, кредит, работать, полгода, впервые, усл...","[970, 9, 43, 786, 1093, 764, 2, 397, 2704, 127..."
9828,1,Почитав отзывы о работе кредитных специалистов...,"[почитать, отзыв, работа, кредитный, специалис...","[1322, 93, 29, 21, 101, 46, 854, 412, 1859, 9,..."
2618,0,"Я напишу свою историю открыто, со всеми цифрам...","[написать, история, открыто, весь, цифра, поня...","[51, 169, 3422, 64, 1083, 473, 944, 4468, 279,..."


### Разделяем метки классов и данные для обучения

Данные для обучения

In [ ]:
x_train_seq = train['Sequences']
y_train = train['Score']

In [ ]:
x_train_seq

idx
13024    [138, 2, 18, 90, 19, 699, 9, 1520, 641, 353, 3...
6213     [153, 2408, 273, 24, 4711, 1972, 39, 15, 115, ...
4859     [13, 2, 107, 920, 14, 414, 729, 91, 52, 137, 7...
10463    [181, 5, 106, 303, 133, 291, 497, 344, 1076, 6...
1006     [39, 999, 17, 203, 449, 29, 2472, 2, 13, 2, 11...
                               ...                        
4070     [77, 44, 2, 674, 691, 397, 438, 4, 446, 9, 8, ...
881      [109, 436, 255, 847, 613, 25, 3944, 5877, 1163...
12884    [1328, 604, 2, 4, 8411, 172, 113, 133, 5874, 1...
3365     [179, 1775, 47, 24, 2498, 6504, 66, 8, 3345, 3...
9930     [525, 39, 472, 283, 101, 1712, 1091, 9860, 972...
Name: Sequences, Length: 11199, dtype: object

In [ ]:
y_train

idx
13024    0
6213     1
4859     1
10463    1
1006     1
        ..
4070     1
881      0
12884    1
3365     1
9930     1
Name: Score, Length: 11199, dtype: int64

Данные для тестирования

In [ ]:
x_test_seq = test['Sequences']
y_test = test['Score']

In [ ]:
x_test_seq

idx
10241    [6, 2007, 6, 996, 5584, 11, 4532, 344, 18, 128...
9612     [5154, 552, 894, 437, 150, 30, 1063, 341, 115,...
2998     [1381, 321, 635, 126, 207, 27, 3793, 80, 3, 17...
11176    [46, 109, 2612, 143, 126, 197, 1315, 43, 1819,...
10527    [468, 1587, 14, 127, 9, 3102, 267, 32, 100, 36...
                               ...                        
6135     [970, 9, 43, 786, 1093, 764, 2, 397, 2704, 127...
9828     [1322, 93, 29, 21, 101, 46, 854, 412, 1859, 9,...
2618     [51, 169, 3422, 64, 1083, 473, 944, 4468, 279,...
12718    [39, 472, 283, 3637, 1830, 161, 848, 468, 1225...
8194     [4183, 284, 318, 170, 6293, 1439, 3957, 215, 3...
Name: Sequences, Length: 2800, dtype: object

In [ ]:
y_test

idx
10241    1
9612     0
2998     0
11176    1
10527    1
        ..
6135     1
9828     1
2618     0
12718    1
8194     1
Name: Score, Length: 2800, dtype: int64

## Создаем мешок слов

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [ ]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [ ]:
x_test = vectorize_sequences(x_test_seq, max_words)

In [ ]:
x_train[0][:100]

array([ 0.,  0., 11.,  0.,  5.,  4.,  1.,  1.,  2.,  2.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  2.,  1.,  2.,  0.,  0.,  2.,  1.,  1.,  1.,
        1.,  0.,  0.,  0.,  2.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,
        1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        1.,  0.,  1.,  0.,  0.,  0.,  2.,  0.,  1.,  2.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  3.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
len(x_train[0])

10000

## Создаем модель машинного обучения

In [ ]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

## Обучаем модель машинного обучения

In [ ]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

## Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [ ]:
lr.score(x_test, y_test)

0.9460714285714286

## Применяем модель для определения тональности отзыва на банк

**Позитивный отзыв**

In [ ]:
positive_text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат.
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж.
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

Подготовка текста к обработке

In [ ]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)

In [ ]:
positive_preprocessed_text

['брать',
 'кредит',
 'мегабанк',
 'автомобиль',
 'выдать',
 'день',
 'никакой',
 'скрытый',
 'комиссия',
 'переплата',
 'банк',
 'удобный',
 'мобильный',
 'приложение',
 'который',
 'быстро',
 'отправить',
 'ежемесячный',
 'платёж',
 'досрочный',
 'гасить',
 'начать',
 'месяц',
 'довольный',
 'оперативность',
 'удобство',
 'огромный',
 'спасибо']

In [ ]:
positive_seq = text_to_sequence(positive_preprocessed_text, word_to_index)

In [ ]:
positive_seq

[170,
 9,
 963,
 136,
 5,
 81,
 1891,
 80,
 1028,
 2,
 287,
 305,
 763,
 8,
 128,
 205,
 489,
 30,
 358,
 1422,
 279,
 34,
 299,
 910,
 1316,
 312,
 75]

In [ ]:
positive_bow = vectorize_sequences([positive_seq], max_words)

In [ ]:
positive_bow[0][0:100]

array([0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Выполняем распознавание

In [ ]:
result = lr.predict(positive_bow)

In [ ]:
result

array([1])

**Негативный отзыв**

In [ ]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [ ]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_seq = text_to_sequence(negative_preprocessed_text, word_to_index)
negative_bow = vectorize_sequences([negative_seq], max_words)

In [ ]:
negative_bow[0][0:100]

array([0., 0., 2., 0., 2., 0., 0., 0., 2., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [ ]:
result = lr.predict(negative_bow)

In [ ]:
result

array([0])

In [ ]:
result = lr.predict_proba(negative_bow)

In [ ]:
result

array([[0.99677403, 0.00322597]])